# Library functions

Here are the library functions re-written using numpy array syntax.

The initialisation phase generates a whole array of random numbers at once, the `if` statement is replaced by a simple `where` and we use the `count` function to compute the number of cars.

For updating the road, we also use a `where` but note that we have used the simpler (commented out) version from the original Python code as a single `where` can only implement a single `if`, `then`, `else` construct. To compute the number of moves we count the number of differences between the new and old roads, but note that we have divide this by two as it counts *both* a car entering and a car leaving a cell.

In [ ]:
# %load trafficlib.py
import sys
import numpy as np

def initroad(road, density, seedval):

    # Here we expect a road without halos

    n = len(road)

    np.random.seed(seedval)

    rng = np.random.random(n)

    road[0:n] = np.where(rng[:] < density, 1, 0)

    ncar = np.sum(road)
    
    return ncar


def updateroad(newroad, oldroad):

    n = len(oldroad)-2

    newroad[1:n+1] = np.where(oldroad[1:n+1]==0, oldroad[0:n], oldroad[2:n+2])

    nmove = (newroad[1:n+1] != oldroad[1:n+1]).sum(dtype=int)
    nmove = nmove / 2

    return nmove


def updatebcs(road):

    n = len(road)-2

    road[0]   = road[n]
    road[n+1] = road[1]


import time

def gettime():

    return time.time()


# Main program

All the main arrays are now numpy arrays - we use 32-bit integers as this is what compiled languages such as C, C++ and Fortran use so it allows for a fair comparison.

It is basically the same as the naive Python code except we can initialise `oldroad` directly using the function call - for lists, taking a subarray creates a new copy of the array whereas for numpy arrays a subarray is still part of the original array.

 * Run the program as-is - is it any faster than the previous code?
 * Reduce the length of the road by factors of ten down to 1024 and note the speed in each case - can you explain the behaviour?
 * Since numpy arrays operations are so much faster, it is important to use array syntax for all operations. How is the speed affected if you replace the copy-back step with a Python loop?

In [ ]:
# %load traffic.py
#!/usr/bin/env python

import sys
import time
import numpy as np

from trafficlib import initroad, updatebcs, updateroad, gettime

def main(argv):

    # Simulation parameters
    seedval = 5743
    ncell = 10240000
    maxiter = 1024000000//ncell
    printfreq = maxiter//10

    newroad  = np.zeros(ncell+2, dtype=np.int32)
    oldroad  = np.zeros(ncell+2, dtype=np.int32)

    density = 0.52

    print(f"Length of road is {ncell}")
    print(f"Number of iterations is {maxiter}")
    print(f"Target density of cars is {density}")

    # Initialise road accordingly using random number generator
    print(f"Initialising ...")

    ncars = initroad(oldroad[1:ncell+1], density, seedval)

    print(f"Actual Density of cars is {format(float(ncars)/float(ncell))}\n")

    tstart = gettime()

    for iter in range(1, maxiter+1):

        updatebcs(oldroad)

        nmove = updateroad(newroad, oldroad)

        # Copy new to old array
        oldroad[1:ncell+1] = newroad[1:ncell+1]

        if iter % printfreq == 0:

          print(f"At iteration {iter} average velocity is {float(nmove)/float(ncars):.6f}")

    tstop = gettime()

    print(f"\nFinished\n")
    print(f"Time taken was {tstop-tstart:.2f} seconds")
    print(f"Update rate was {1.0e-6*ncell*maxiter/(tstop-tstart):.2f} MCOPs")

if __name__ == "__main__":
    main(sys.argv[1:])
